In [3]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json
import requests
import datetime
import os

In [4]:
import requests

url = "https://kauth.kakao.com/oauth/token"

#data = {
#    "grant_type" : "authorization_code",
#    "client_id" : "REST_API_키",
#    "redirect_uri" : "https://localhost.com",
#    "code" : "발급받은 토큰값"
#}

data = {
    "grant_type" : "authorization_code",
    "client_id" : "",
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

{'access_token': '7wGtEmpDFGiziMbjMx2fjLPeOKE0TG4pWYaYtwo9dRsAAAF8_od0tw', 'token_type': 'bearer', 'refresh_token': 'zfgGxyEg6-xDFBvyn_DA4CX1Ze2aQkyPj8FT-Ao9dRsAAAF8_od0tw', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}


In [5]:
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"
APP_KEY_SAVED_FILE = "res/kakao_msg/app_key"

In [6]:
def save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens):
    with open(KAKAO_TOKEN_SAVED_FILE, "w") as fp:
        json.dump(tokens, fp)

In [7]:
save_tokens(KAKAO_TOKEN_SAVED_FILE,tokens)

In [8]:
# 초기 한 번만 수행함(REST APP KEY 저장 목적)
import pickle

app_key = ""

f = open(APP_KEY_SAVED_FILE, 'wb')
pickle.dump(app_key, f)
f.close()

In [22]:
# REST APP KEY 저장 잘 되었는지 확인하는 목적
f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

print()

In [10]:
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [11]:
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "." + now
        os.rename(filename, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

In [21]:
app = Flask(__name__)
CORS(app)

@app.route('/spring2pythonRequestWithData', methods = ['GET', 'POST'])
def spring2pythonRequestWithData():
    
    params = json.loads(request.get_data(), encoding='utf-8')
    print("params: ", params)
    
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_string = ""
    name = ""
    board_no = ""
    
    ##################### 개조 구간 시작 #####################
    #params_string = ""
    #for key in params.keys():
    #    params_string += 'key: {}, value: {}<br>'.format(key, params[key])
    
    values = []
    for key in params.keys():
        values.append(params[key])
        
    print("values:", values)
    board_no = values[0]
    name =  values[1]
    
    print("게시판번호:", board_no)
    print("작성자:", name)
        
    
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)
    print("tokens: ", tokens)

    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : "글쓴이{1}의{0}번 게시글이 신고되었습니다 확인부탁드립니다.".format(board_no,name),
            "link" : {
                "web_url" : "http://localhost:8080/boardReadPage/{0}".format(board_no)
            }
        })
    }

    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
    
    if response.status_code == 401:
        tokens = update_tokens(app_key, KAKAO_TOKEN_SAVED_FILE)
        
    print("tokens: ", tokens)
    
    ##################### 개조 구간 끄읕 #####################
    
    return jsonify(params_string)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
<ipython-input-21-f84e7b90315b>:7: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')
127.0.0.1 - - [08/Nov/2021 17:05:00] "POST /spring2pythonRequestWithData HTTP/1.1" 200 -


params:  {'boardNo': 13, 'memberId': 'tkddn1212'}
values: [13, 'tkddn1212']
게시판번호: 13
작성자: tkddn1212
tokens:  {'access_token': '7wGtEmpDFGiziMbjMx2fjLPeOKE0TG4pWYaYtwo9dRsAAAF8_od0tw', 'token_type': 'bearer', 'refresh_token': 'zfgGxyEg6-xDFBvyn_DA4CX1Ze2aQkyPj8FT-Ao9dRsAAAF8_od0tw', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}
200
tokens:  {'access_token': '7wGtEmpDFGiziMbjMx2fjLPeOKE0TG4pWYaYtwo9dRsAAAF8_od0tw', 'token_type': 'bearer', 'refresh_token': 'zfgGxyEg6-xDFBvyn_DA4CX1Ze2aQkyPj8FT-Ao9dRsAAAF8_od0tw', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}
